# Rotational dynamics Quickstart

Calculation of rotational dynamics proceeds in two steps. At the first step, we obtain molecular field-free rotational energies, wave functions, and matrix elements of desired electric and magnetic property tensors, such as, for example, dipole moment or polarizability. At the second step, we solve the time-dependent problem with applied external electric and magnetic fields using the field-free solutions, obtained at the first step, as a basis.

## Molecular field-free rotational states

In [1]:
from richmol.rot.molecule import Molecule, mol_tensor
from richmol.rot.solution import solve

There are three common ways to define molecule and its properties using calculated or experimental data or combination of both:

1. Provide Cartesian coordinates of atoms and molecular property tensors, referring to the same coordinate frame, obtained, for example, from quantum chemical calculations.

2a. Provide spectroscopic constants ($A$, $B$, $C$, $\Delta_{J}$, $\Delta_{J,K}$ ...) and molecular property tensors in the coordinate frame of principal axes of inertia (PAI).

2b. Provide spectroscopic constants ($A$, $B$, $C$, $\Delta_{J}$, $\Delta_{J,K}$ ...) and molecular property tensors which are not in the PAI frame. In this case, the Cartesian coordinates of atoms, referring to the same frame as property tensors, must be provided. These will be used only for rotating the property tensors to the PAI frame.

### Molecular geometry and property tensors

Here is an example for water molecule, using data obtained from a quantum chemical calculation

In [2]:
water = Molecule()

# Cartesian coordinates of atoms
water.XYZ = ("bohr",
             "O",  0.00000000,   0.00000000,   0.12395915,
             "H",  0.00000000,  -1.43102686,  -0.98366080,
             "H",  0.00000000,   1.43102686,  -0.98366080)

# dipole moment (au)
water.dip = [0, 0, -0.7288]

# polarizability tensor (au)
water.pol = [[9.1369, 0, 0], [0, 9.8701, 0], [0, 0, 9.4486]]

print("Masses and Cartesian coordinates of atoms")
for atom in water.XYZ:
    print(atom['label'], atom['mass'], atom['xyz'])

Masses and Cartesian coordinates of atoms
O 15.99491462 [0.         0.         0.06559636]
H 1.0078250322 [ 0.         -0.7572668  -0.52053088]
H 1.0078250322 [ 0.          0.7572668  -0.52053088]


By default, calculations are carried out for the main isotopologue. To specify a non-standard isotope, put the corresponding isotope number next to the atom label, for example, "O18" for oxygen-18 or "H2" for deuterium

In [3]:
# example of D2^{18}O
D2_18O = Molecule()
D2_18O.XYZ = ("bohr",
             "O18",  0.00000000,   0.00000000,   0.12395915,
             "H2",   0.00000000,  -1.43102686,  -0.98366080,
             "H2",   0.00000000,   1.43102686,  -0.98366080)

print("Masses and Cartesian coordinates of atoms")
for atom in D2_18O.XYZ:
    print(atom['label'], atom['mass'], atom['xyz'])

Masses and Cartesian coordinates of atoms
O18 17.999159613 [0.         0.         0.06559636]
H2 2.0141017781 [ 0.         -0.7572668  -0.52053088]
H2 2.0141017781 [ 0.          0.7572668  -0.52053088]


We can also read/store the geometry from/to XYZ file

In [4]:
# store Cartesian coordinates of atoms into XYZ file
water.store_xyz("water.xyz", "some comment line")

# read Cartesian coordinates of atoms from XYZ file
water2 = Molecule()
water2.XYZ = "water.xyz"

print("Masses and Cartesian coordinates of atoms")
for atom in water2.XYZ:
    print(atom['label'], atom['mass'], atom['xyz'])

Masses and Cartesian coordinates of atoms
O 15.99491462 [0.         0.         0.06559636]
H 1.0078250322 [ 0.         -0.7572668  -0.52053088]
H 1.0078250322 [ 0.          0.7572668  -0.52053088]


The molecular frame embedding, i.e., the orientation of $x,y,z$ axes with respect to molecule, can be altered using `frame`.
The molecule properties `dip` for the dipole moment and `pol` for the polarizability tensor (and few others) will be dynamically rotated to a new coordinate frame whenever the latter is changed

In [5]:
# change frame to inertial principal axes system (ipas)
water.frame = "ipas"

# or equivalently
water.frame = "diag(inertia)" # frame where inertia tensor is diagonal

print("coordinates of atoms\n", water.XYZ)
print("dipole moment\n", water.dip)
print("polarizability\n", water.pol)

print("inertia tensor\n", water.inertia)

coordinates of atoms
 [('O', 15.99491462, [ 0.        ,  0.06559636,  0.        ])
 ('H',  1.00782503, [-0.7572668 , -0.52053088,  0.        ])
 ('H',  1.00782503, [ 0.7572668 , -0.52053088,  0.        ])]
dipole moment
 [ 0.     -0.7288  0.    ]
polarizability
 [[9.8701 0.     0.    ]
 [0.     9.4486 0.    ]
 [0.     0.     9.1369]]
inertia tensor
 [[ 0.61496944 -0.         -0.        ]
 [-0.          1.15588059 -0.        ]
 [-0.         -0.          1.77085002]]


Multiple frame operations can be combined together, for example, we can rotate to the inertial principal axes system and then permute $x$ and $z$ axes

In [6]:
water.frame = "ipas"
water.frame = "zyx"

# or, equivalently in one line (with operatiorns performed form right to left)
water.frame = "zyx,ipas"

print("coordinates of atoms\n", water.XYZ)
print("dipole moment\n", water.dip)
print("polarizability\n", water.pol)

print("inertia tensor\n", water.inertia)

coordinates of atoms
 [('O', 15.99491462, [ 0.        ,  0.06559636,  0.        ])
 ('H',  1.00782503, [ 0.        , -0.52053088, -0.7572668 ])
 ('H',  1.00782503, [ 0.        , -0.52053088,  0.7572668 ])]
dipole moment
 [ 0.     -0.7288  0.    ]
polarizability
 [[9.1369 0.     0.    ]
 [0.     9.4486 0.    ]
 [0.     0.     9.8701]]
inertia tensor
 [[ 1.77085002 -0.         -0.        ]
 [-0.          1.15588059 -0.        ]
 [-0.         -0.          0.61496944]]


The principal axes system can be defined with respect to any rank-2 symmetric matrix. In many cases it is convenient to choose molecular frame such that polarizability tensor becomes diagonal

In [7]:
water.frame = "diag(pol)" # can also be combined with axes permutations, e.g., "xzy,diag(pol)"

print("coordinates of atoms\n", water.XYZ)
print("dipole moment\n", water.dip)
print("polarizability\n", water.pol)


coordinates of atoms
 [('O', 15.99491462, [ 0.        ,  0.06559636,  0.        ])
 ('H',  1.00782503, [ 0.        , -0.52053088, -0.7572668 ])
 ('H',  1.00782503, [ 0.        , -0.52053088,  0.7572668 ])]
dipole moment
 [ 0.     -0.7288  0.    ]
polarizability
 [[9.1369 0.     0.    ]
 [0.     9.4486 0.    ]
 [0.     0.     9.8701]]


We can also define custom rotation matrix or principal axis matrix, as demonstrated below

In [8]:
import numpy as np
import scipy

# generate random matrix
mat = np.random.rand(3,3)

# random rotation matrix
water.custom_rot = scipy.linalg.expm((mat - mat.T)/2)

# random symmetric matrix
water.custom_pam = (mat + mat.T)/2

# use 'custom_rot' (orthogonal) matrix to rotate the frame
water.frame = "custom_rot"

# alternatively use 'custom_pam' (symmetric) matrix as principal axes matrix
water.frame = "diag(custom_pam)"

Multiple occurences of `frame` assignments lead to the accumulation of the corresponding frame transformations. Use `frame`=None or `frame`="None" to reset the frame to the one defined by the input Cartesian coordinates of atoms

In [20]:
water.frame = "diag(inertia)" # rotate to ipas
water.frame = "zxy" # then permute axes (123)

# now we want to permute axes (23) in the original frame, reset frame rotation
water.frame = None
water.frame = "xzy"

# or equivalently in one line
#water.frame = "xzy,None"

print(water.XYZ)

[('O', 15.99491462, [ 0.        ,  0.06559636,  0.        ])
 ('H',  1.00782503, [ 0.        , -0.52053088, -0.7572668 ])
 ('H',  1.00782503, [ 0.        , -0.52053088,  0.7572668 ])]


Only certain molecule properties (such as `dip`, `pol`, `inertia`, `XYZ`, and few others) are automatically rotated when the frame is altered. This is demonstrated in the following example

In [21]:
water.vec = [1,2,3]
water.mat = [[1,2,3],[4,5,6],[7,8,9]]

water.frame = "zyx,null"

# axes permutation (13) does not affect vec and mat, but dip
print("vec\n", water.vec)
print("mat\n", water.mat)
print("dip\n", water.dip)

vec
 [1, 2, 3]
mat
 [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
dip
 [-0.7288  0.      0.    ]


If we want to declare custom Cartesian tensor which is dynamically rotated to a new frame, we use `mol_tensor` function

In [22]:
water.vec = mol_tensor([1,2,3])
water.mat = mol_tensor([[1,2,3],[4,5,6],[7,8,9]])

water.frame = "zyx,null"

# axes permutation (13) now affects also vec and mat
print("vec\n", water.vec)
print("mat\n", water.mat)
print("dip\n", water.dip)

# we can then change vec and mat values, they will still be dynamically rotated
water.vec = [7,8,9]
water.mat = [[11,12,13],[14,15,16],[17,18,19]]

print("\nnew vec\n", water.vec)
print("new mat\n", water.mat)

vec
 [3. 2. 1.]
mat
 [[9. 8. 7.]
 [6. 5. 4.]
 [3. 2. 1.]]
dip
 [-0.7288  0.      0.    ]

new vec
 [9. 8. 7.]
new mat
 [[19. 18. 17.]
 [16. 15. 14.]
 [13. 12. 11.]]


### Rotational constants

The rotational constants (in units of cm$^{-1}$) can be calculated from the input molecular geometry using `ABC`. This will work only if the molecular frame is set to the inertia principal axes system or if the inertia tensor is diagonal
!!!! define constants MHz, GHz to convert units to cm^-1 or MHz_to_invcm() !!!!!

In [25]:
water.frame = "diag(pol)"
print(water.ABC) # works because inertia tensor is diagonal in the polarizability principal axes system of water

water.frame = "diag(inertia)"
print(water.ABC) # the constants in inertia frame will be sorted in ascending order

[9.51951209577557, 14.584229929162115, 27.41213978457678]
[27.41213978457678, 14.584229929162115, 9.51951209577557]


The values of rotational constants can also be user specified, for example, using the experimental values. In these case, `ABC` will return the user-defined values. If we want to access rotational constants calculated from the molecular geometry, we should use `ABC_calc` !!! ABC_geom !!!

In [26]:
print("calculated ABC", water.ABC)
water.ABC = (9.285, 14.512, 27.877) # user-defined rotational constants in units of cm^-1
print("user input ABC ", water.ABC)
print("calculated ABC", water.ABC_calc)

calculated ABC [27.41213978457678, 14.584229929162115, 9.51951209577557]
user input ABC  [27.877, 14.512, 9.285]
calculated ABC [27.41213978457678, 14.584229929162115, 9.51951209577557]


***************************

**Important**: If user-defined rotational constants have been specified, the rotational Hamiltonian is built using these values

***************************

For linear and spherical-top molecules, use `B` to print or assign the rotational constant

In [27]:
ocs = Molecule()
ocs.B = 0.20286 # experimental value
print(ocs.B)

ocs.XYZ = ("angstrom", "C", 0, 0, 0, "S", 0, 0, -1.56, "O", 0, 0, 1.16)
print(ocs.B_calc) # calculated value

print(ocs.linear()) # True if molecule is linear

0.20286
0.20317858059669255
True


If molecular geometry is specified, the input rotational constants are always checked to agree within 5% difference with the calculated values. Here is an example of an error produced when molecular frame is not in the inertia principal axes system

In [29]:
water2 = Molecule()
water2.ABC = (9.285, 14.512, 27.877) # user-defined rotational constants in units of cm^-1

water2.XYZ = ("bohr",
              "O",  0.00000000,   0.00000000,   0.12395915,
              "H",  0.00000000,  -1.43102686,  -0.98366080,
              "H",  0.00000000,   1.43102686,  -0.98366080)

# polarizability tensor (au)
water2.pol = [[9.1369, 0, 0], [0, 9.8701, 0], [0, 0, 9.4486]]

water2.frame = 'ipas' # inertia principal axes system
print("inertia frame")
print("calculated ABC", water2.ABC_calc)
print("input ABC     ", water2.ABC)

water2.frame = 'diag(pol)' # polarizability principal axes system
print("\npolarizability frame")
print("calculated ABC", water2.ABC_calc)
print("input ABC     ", water2.ABC) # specified rotaitonal constants disagree much with the geometry 

inertia frame
calculated ABC [27.41213978457678, 14.584229929162115, 9.51951209577557]
input ABC      [27.877, 14.512, 9.285]

polarizability frame
calculated ABC [9.51951209577557, 14.584229929162115, 27.41213978457678]


ValueError: input experimental rotational constants differ much from the calculated once
        exp          calc       exp-calc
A    27.877000     9.519512    18.357488
B    14.512000    14.584230    -0.072230
C     9.285000    27.412140   -18.127140

### Rotational solutions 

The rotational energies and wave functions can be calculated for a specified range of $J$ quantum number using function `solve`

In [30]:
sol = solve(water, Jmin=0, Jmax=10)

Returned object is a dictionary `sol[J][sym]` containing solutions for different values of $J$ and different symetries (of $C_1$ group by default). The state energies (in cm$^{-1}$) and assignments (by $J, k, \tau$ quantum numbers) can be printed out as following

In [31]:
print("J  sym #    energy      J   k  tau  |leading coef|^2")
for J, sol_J in sol.items():
    for symmetry, sol_sym in sol_J.items():
        for istate in range(sol_sym.nstates):
            print(J, "%4s"%symmetry, istate, "%12.6f"%sol_sym.enr[istate], sol_sym.assign[istate])

J  sym #    energy      J   k  tau  |leading coef|^2
0    A 0     0.000000 ['0' '0' '0' ' 1.000000']
1    A 0    23.797000 ['1' '0' '1' ' 1.000000']
1    A 1    37.162000 ['1' '1' '0' ' 1.000000']
1    A 2    42.389000 ['1' '1' '1' ' 1.000000']
2    A 0    70.133328 ['2' '0' '0' ' 0.981068']
2    A 1    79.529000 ['2' '1' '1' ' 1.000000']
2    A 2    95.210000 ['2' '1' '0' ' 1.000000']
2    A 3   135.305000 ['2' '2' '1' ' 1.000000']
2    A 4   136.562672 ['2' '2' '0' ' 0.981068']
3    A 0   136.909477 ['3' '0' '1' ' 0.922382']
3    A 1   142.369084 ['3' '1' '0' ' 0.995098']
3    A 2   173.535239 ['3' '1' '1' ' 0.992048']
3    A 3   206.696000 ['3' '2' '0' ' 1.000000']
3    A 4   212.568523 ['3' '2' '1' ' 0.922382']
3    A 5   287.298916 ['3' '3' '0' ' 0.995098']
3    A 6   287.494761 ['3' '3' '1' ' 0.992048']
4    A 0   222.369478 ['4' '0' '0' ' 0.834399']
4    A 1   225.067555 ['4' '1' '1' ' 0.982779']
4    A 2   276.014787 ['4' '1' '0' ' 0.962955']
4    A 3   300.891698 ['4' '2' '1' 

The energies are in units of cm$^{-1}$ and the assignment is a list of numbers ($J$, $k$, $\tau$, $c$) denoting the leading symmetric-top function in the expansion of wave function. $\tau$ is the pairty of rotational state, defined as $(-1)^\tau$, and $c$ is the absolute value of the leading coefficient.

We can print more than one leading contribution in the assignment

In [18]:
print("J  sym #    energy      J   k  tau  |leading coef|^2")
for J,sol_J in sol.items():
    for symmetry,sol_sym in sol_J.items():
        sol_sym.assign_nprim = 3 # print up to three leading contributions
        for istate in range(sol_sym.nstates):
            print(J, "%4s"%symmetry, istate, "%12.6f"%sol_sym.enr[istate], sol_sym.assign[istate])

J  sym #    energy      J   k  tau  |leading coef|^2
0    A 0     0.000000 ['0' '0' '0' ' 1.000000']
1    A 0    23.797000 ['1' '0' '1' ' 1.000000' '1' '1' '0' ' 0.000000' '1' '1' '1' ' 0.000000']
1    A 1    37.162000 ['1' '1' '0' ' 1.000000' '1' '0' '1' ' 0.000000' '1' '1' '1' ' 0.000000']
1    A 2    42.389000 ['1' '1' '1' ' 1.000000' '1' '0' '1' ' 0.000000' '1' '1' '0' ' 0.000000']
2    A 0    70.133328 ['2' '0' '0' ' 0.981068' '2' '2' '0' ' 0.018932' '2' '1' '0' ' 0.000000']
2    A 1    79.529000 ['2' '1' '1' ' 1.000000' '2' '1' '0' ' 0.000000' '2' '0' '0' ' 0.000000']
2    A 2    95.210000 ['2' '1' '0' ' 1.000000' '2' '1' '1' ' 0.000000' '2' '2' '0' ' 0.000000']
2    A 3   135.305000 ['2' '2' '1' ' 1.000000' '2' '0' '0' ' 0.000000' '2' '1' '0' ' 0.000000']
2    A 4   136.562672 ['2' '2' '0' ' 0.981068' '2' '0' '0' ' 0.018932' '2' '1' '0' ' 0.000000']
3    A 0   136.909477 ['3' '0' '1' ' 0.922382' '3' '2' '1' ' 0.077618' '3' '1' '0' ' 0.000000']
3    A 1   142.369084 ['3' '1' '0' 

### Symmetry

The molecular symmetry can be specified using molecule property `sym`, for example, the above calculation can be done using the $D_2$ or $C_{2v}$ rotational symmetry groups

In [33]:
water.sym = 'D2'
sol_d2 = solve(water, Jmin=0, Jmax=3)

water.sym = 'C2v'
sol_c2v = solve(water, Jmin=0, Jmax=3)

print("D2 solutions")
print("J  sym #    energy      J   k  tau  |leading coef|^2")
for J,sol_J in sol_d2.items():
    for symmetry,sol_sym in sol_J.items():
        for istate in range(sol_sym.nstates):
            print(J, "%4s"%symmetry, istate, "%12.6f"%sol_sym.enr[istate], sol_sym.assign[istate])

print("C2v solutions")
print("J  sym #    energy      J   k  tau  |leading coef|^2")
for J,sol_J in sol_c2v.items():
    for symmetry,sol_sym in sol_J.items():
        for istate in range(sol_sym.nstates):
            print(J, "%4s"%symmetry, istate, "%12.6f"%sol_sym.enr[istate], sol_sym.assign[istate])

D2 solutions
J  sym #    energy      J   k  tau  |leading coef|^2
0    A 0     0.000000 ['0' '0' '0' ' 1.000000']
1   B1 0    23.797000 ['1' '0' '1' ' 1.000000']
1   B2 0    37.162000 ['1' '1' '0' ' 1.000000']
1   B3 0    42.389000 ['1' '1' '1' ' 1.000000']
2    A 0    70.133328 ['2' '0' '0' ' 0.981068']
2    A 1   136.562672 ['2' '2' '0' ' 0.981068']
2   B1 0   135.305000 ['2' '2' '1' ' 1.000000']
2   B2 0    95.210000 ['2' '1' '0' ' 1.000000']
2   B3 0    79.529000 ['2' '1' '1' ' 1.000000']
3    A 0   206.696000 ['3' '2' '0' ' 1.000000']
3   B1 0   136.909477 ['3' '0' '1' ' 0.922382']
3   B1 1   212.568523 ['3' '2' '1' ' 0.922382']
3   B2 0   142.369084 ['3' '1' '0' ' 0.995098']
3   B2 1   287.298916 ['3' '3' '0' ' 0.995098']
3   B3 0   173.535239 ['3' '1' '1' ' 0.992048']
3   B3 1   287.494761 ['3' '3' '1' ' 0.992048']
C2v solutions
J  sym #    energy      J   k  tau  |leading coef|^2
0   A1 0     0.000000 ['0' '0' '0' ' 1.000000']
1   A2 0    23.797000 ['1' '0' '1' ' 1.000000']
1  

We can restrict solutions to have only certain symmetries, by specifying a list of desired symmetries for different values of $J$ in `only` argument to `solve`

In [34]:
Jmin = 0
Jmax = 5

water.sym = 'D2'
sym = {J : ['B2', 'B1'] for J in range(Jmin, Jmax+1)} # list of desired symmetries for different J
sol_d2 = solve(water, Jmin=0, Jmax=3, only={'sym':sym})

water.sym = 'C2v'
sym = {J : ['A2', 'B2'] for J in range(Jmin, Jmax+1)}
sol_c2v = solve(water, Jmin=0, Jmax=3, only={'sym':sym})

print("D2 solutions")
print("J  sym #    energy      J   k  tau  |leading coef|^2")
for J,sol_J in sol_d2.items():
    for symmetry,sol_sym in sol_J.items():
        for istate in range(sol_sym.nstates):
            print(J, "%4s"%symmetry, istate, "%12.6f"%sol_sym.enr[istate], sol_sym.assign[istate])

print("C2v solutions")
print("J  sym #    energy      J   k  tau  |leading coef|^2")
for J,sol_J in sol_c2v.items():
    for symmetry,sol_sym in sol_J.items():
        for istate in range(sol_sym.nstates):
            print(J, "%4s"%symmetry, istate, "%12.6f"%sol_sym.enr[istate], sol_sym.assign[istate])


D2 solutions
J  sym #    energy      J   k  tau  |leading coef|^2
1   B1 0    23.797000 ['1' '0' '1' ' 1.000000']
1   B2 0    37.162000 ['1' '1' '0' ' 1.000000']
2   B1 0   135.305000 ['2' '2' '1' ' 1.000000']
2   B2 0    95.210000 ['2' '1' '0' ' 1.000000']
3   B1 0   136.909477 ['3' '0' '1' ' 0.922382']
3   B1 1   212.568523 ['3' '2' '1' ' 0.922382']
3   B2 0   142.369084 ['3' '1' '0' ' 0.995098']
3   B2 1   287.298916 ['3' '3' '0' ' 0.995098']
C2v solutions
J  sym #    energy      J   k  tau  |leading coef|^2
1   A2 0    23.797000 ['1' '0' '1' ' 1.000000']
1   B2 0    42.389000 ['1' '1' '1' ' 1.000000']
2   A2 0   135.305000 ['2' '2' '1' ' 1.000000']
2   B2 0    79.529000 ['2' '1' '1' ' 1.000000']
3   A2 0   136.909477 ['3' '0' '1' ' 0.922382']
3   A2 1   212.568523 ['3' '2' '1' ' 0.922382']
3   B2 0   173.535239 ['3' '1' '1' ' 0.992048']
3   B2 1   287.494761 ['3' '3' '1' ' 0.992048']


### Centrifugal distortion rotational constants

The centrifugal distortion constants are used to build the Watson-type asymmetric top Hamiltonian in the A or S standard reduced form (J. K. G. Watson in "Vibrational Spectra and Structure" (Ed: J. Durig) Vol 6 p 1, Elsevier, Amsterdam, 1977)

* A-form: $H_A = H_\text{rigrot} - \Delta_{J} J^{4} - \Delta_{JK} J^{2} J_{z}^{2} - \Delta_{K} J_{z}^{4} - \frac{1}{2}  [ \delta_{J} J^{2} + \delta_{K} J_{z}^{2}, J_{+}^{2} + J_{-}^{2} ]_{+} + H_{J} J^{6} + H_{JK} J^{4} J_{z}^{2} + H_{KJ} J^{2} J_{z}^{4} + H_{K} J_{z}^{6} + \frac{1}{2} [ \phi_{J} J^{4} + \phi_{JK} J^{2} J_{z}^{2} + \phi_{K} J_{z}^{4}, J_{+}^{2} + J_{-}^{2} ]_{+}$

* S-form: $H_S = H_\text{rigrot} - \Delta_{J} J^{4} - \Delta_{JK} J^{2} J_{z}^{2} - \Delta_{K} J_{z}^{4} + d_{1} J^{2} (J_{+}^{2} + J_{-}^{2}) + d_{2} (J_{+}^{4} + J_{-}^{4}) + H_{J} J^{6} + H_{JK} J^{4} J_{z}^{2} + H_{KJ} J^{2} J_{z}^{4} + H_{K} J_{z}^{6} + h_{1} J^{4} (J_{+}^{2} + J_{-}^{2}) + h_{2} J^{2} (J_{+}^{4} + J_{-}^{4}) + h_{3} (J_{+}^{6} + J_{-}^{6})$

The type of Watson Hamiltonian can be specified using `watson` molecule property, while the centrifugal constants are defined as molecule attributes `DeltaJ`, `DeltaJK`, `DeltaK`, `deltaJ`, `deltaK`, `HJ`, `HJK`, `HKJ`, `HK`, `phiJ`, `phiJK`, `phiK`, `d1`, `d2`, `h1`, `h2`, `h3`.

In [39]:
from richmol.constants import vellgt
MHz_to_invcm = 1/vellgt * 1e6

# NIST values of rotational constants for H2O

water = Molecule()
water.ABC = [val * MHz_to_invcm for val in (835840.288, 435351.717, 278138.700)]

water.watson = 'watson_a' # or 'watson_s' for S-type

water.DeltaJ = 37.59422 * MHz_to_invcm
water.DeltaJK = -172.9128 * MHz_to_invcm
water.DeltaK = 973.29052 * MHz_to_invcm
water.deltaJ = 15.210402 * MHz_to_invcm
water.deltaK = 41.0502 * MHz_to_invcm
water.HJ = 1.56556e-2 * MHz_to_invcm
water.HJK = -4.2081e-2 * MHz_to_invcm
water.HKJ = -5.09508e-1 * MHz_to_invcm
water.HK = 3.733028 * MHz_to_invcm
water.phiJ = 7.79579e-3 * MHz_to_invcm
water.phiJK = -2.5165e-2 * MHz_to_invcm
water.phiK = 1.0971 * MHz_to_invcm

water.sym = "C2v"

sol = solve(water, Jmin=0, Jmax=5, verbose=True) # set verbose=True to check which attributes of water are used to build the Watson Hamiltonian

print("\nJ  sym #    energy      J   k  tau  |leading coef|^2")
for J,sol_J in sol.items():
    for symmetry,sol_sym in sol_J.items():
        for istate in range(sol_sym.nstates):
            print(J, "%4s"%symmetry, istate, "%12.6f"%sol_sym.enr[istate], sol_sym.assign[istate])

solve for J = 0 and symmetry A1
build rigid-rotor Hamiltonian from rotational constants, A, B, C = (27.88063093968828, 14.521770157406696, 9.277708380509026) cm-1
add 'watson_s' term 'DeltaJ' = 0.0012540081978980272
add 'watson_s' term 'DeltaJK' = -0.005767750168017903
add 'watson_s' term 'DeltaK' = 0.03246547716687389
add 'watson_s' term 'deltaJ' = 0.0005073643980730162
add 'watson_s' term 'deltaK' = 0.001369287282070318
add 'watson_s' term 'HJ' = 5.222146048784189e-07
add 'watson_s' term 'HJK' = -1.4036710690033436e-06
add 'watson_s' term 'HKJ' = -1.6995357501622004e-05
add 'watson_s' term 'HK' = 0.0001245204107169367
add 'watson_s' term 'phiJ' = 2.6003956377048017e-07
add 'watson_s' term 'phiJK' = -8.394140455661496e-07
add 'watson_s' term 'phiK' = 3.659531688418926e-05
solve for J = 1 and symmetry A2
build rigid-rotor Hamiltonian from rotational constants, A, B, C = (27.88063093968828, 14.521770157406696, 9.277708380509026) cm-1
add 'watson_s' term 'DeltaJ' = 0.0012540081978980272


### Matrix elements of laboratory-frame Cartesian tensor operators

Once we have obtained the solutions of the field-free rotational problem (using `solve`), we can compute the matrix elements of various laboratory-frame molecule-field interaction tensors and observables

In [46]:
from richmol.rot.labtens import LabTensor

Jmin = 0
Jmax = 5
water.sym = 'D2'

sol = solve(water, Jmin=Jmin, Jmax=Jmax)

# expectation values of lab-frame dipole moment operator
DIP = LabTensor(water.dip, sol)

# expectation values of lab-frame polarizability tensor
POL = LabTensor(water.pol, sol)

In [50]:
dx = DIP.tomat(form='full', cart='x') # add csr, coo format
print(dx)

[[0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 ...
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]]


### Store field-free data in richmol database file

In [51]:
from richmol.rot import rchm # will be moved later from rot to richmol
from richmol.rot.molecule import Molecule
from richmol.rot.solution import solve, H0Tensor
from richmol.rot.labtens import LabTensor
from richmol.constants import vellgt

MHz_to_invcm = 1/vellgt * 1e6

water = Molecule()

# Cartesian coordinates of atoms
water.XYZ = ("bohr",
             "O",  0.00000000,   0.00000000,   0.12395915,
             "H",  0.00000000,  -1.43102686,  -0.98366080,
             "H",  0.00000000,   1.43102686,  -0.98366080)

# dipole moment (au)
water.dip = [0, 0, -0.7288]

# polarizability tensor (au)
water.pol = [[9.1369, 0, 0], [0, 9.8701, 0], [0, 0, 9.4486]]

water.frame = 'diag(inertia)'

water.ABC = [val * MHz_to_invcm for val in (835840.288, 435351.717, 278138.700)]

water.sym = "C2v"

# store molecular data
rchm.store('water.h5', water, replace=True, comment="This is data for water molecule caluclated at CCSD(T)/aug-cc-pVTZ level of theory")

# solve for J=0..5
sol = solve(water, Jmin=0, Jmax=5)

# matrix elements of lab-frame dipole moment vector
DIP = LabTensor(water.dip, sol)

# matrix elements of lab-frame polarizability tensor
POL = LabTensor(water.pol, sol)

# store fiueld-free solutions
#rchm.store('water.h5', H0Tensor(water, sol))

# store lab-frame tensors
rchm.store('water.h5', DIP, replace=True, comment="values are in atomic units") # to do: to be able to ammend for different pairs of J quanta
rchm.store('water.h5', POL, replace=True, comment="values are in atomic units")

In [52]:
# read molecular data from file
mol = rchm.get_molecule('water.h5')

print('attributes', mol.__name__)
for elem in vars(mol):
    if not elem.startswith('__'):
        print('\n', elem)
        print(getattr(mol, elem))


attributes molecule

 ABC
[27.88063094 14.52177016  9.27770838]

 comment
b'This is data for water molecule caluclated at CCSD(T)/aug-cc-pVTZ level of theory'

 date
2021-03-31 15:02:46

 dip
[ 0.     -0.7288  0.    ]

 inertia
[[ 0.61496944 -0.         -0.        ]
 [-0.          1.15588059 -0.        ]
 [-0.         -0.          1.77085002]]

 ipas
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

 label
[b'O' b'H' b'H']

 mass
[15.99491462  1.00782503  1.00782503]

 mat
[[15. 16. 14.]
 [18. 19. 17.]
 [12. 13. 11.]]

 pol
[[9.8701 0.     0.    ]
 [0.     9.4486 0.    ]
 [0.     0.     9.1369]]

 vec
[8. 9. 7.]

 xyz
[[ 0.          0.06559636  0.        ]
 [-0.7572668  -0.52053088  0.        ]
 [ 0.7572668  -0.52053088  0.        ]]


In [53]:
# inspect data groups in file
data = rchm.inspect_file('water.h5')

print("list of data groups")
for key, val in data.items():
    print(key, val)

print("\nlist of attributes for POL")
for elem in vars(data['POL']):
    if not elem.startswith('__'):
        print(elem)

print("\nPOL tensor rank:", data['POL'].rank)
print("POL tensor list of Cartesian components:", data['POL'].cart)
print("POL tensor list of coupled J quanta:", data['POL'].Jpairs)
print("POL tensor list of coupled symmetries:")
for key, val in data['POL'].sympairs.items():
    print("(J1, J2) =", key, "sym:", val)


list of data groups
DIP <class 'richmol.rot.rchm.DIP'>
POL <class 'richmol.rot.rchm.POL'>
molecule <class 'richmol.rot.rchm.molecule'>
obj <class 'richmol.rot.rchm.obj'>

list of attributes for POL
Us
Ux
cart
comment
date
os
rank
tens_flat
Jpairs
sympairs

POL tensor rank: 2
POL tensor list of Cartesian components: ['xx' 'xy' 'xz' 'yx' 'yy' 'yz' 'zx' 'zy' 'zz']
POL tensor list of coupled J quanta: [(0.0, 0.0), (0.0, 2.0), (1.0, 1.0), (1.0, 2.0), (1.0, 3.0), (2.0, 0.0), (2.0, 1.0), (2.0, 2.0), (2.0, 3.0), (2.0, 4.0), (3.0, 1.0), (3.0, 2.0), (3.0, 3.0), (3.0, 4.0), (3.0, 5.0), (4.0, 2.0), (4.0, 3.0), (4.0, 4.0), (4.0, 5.0), (5.0, 3.0), (5.0, 4.0), (5.0, 5.0)]
POL tensor list of coupled symmetries:
(J1, J2) = (0.0, 0.0) sym: [('A1', 'A1')]
(J1, J2) = (0.0, 2.0) sym: [('A1', 'A1'), ('A1', 'A2'), ('A1', 'B1'), ('A1', 'B2')]
(J1, J2) = (1.0, 1.0) sym: [('A2', 'A2'), ('A2', 'B1'), ('A2', 'B2'), ('B1', 'A2'), ('B1', 'B1'), ('B1', 'B2'), ('B2', 'A2'), ('B2', 'B1'), ('B2', 'B2')]
(J1, J2) = (1.0